# Filtering Microcap Alts

In [1]:
#%% Importing modules and data
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import quandl as qd
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from pandas_datareader import data
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import os
import glob
from time import sleep


# Functions for pulling from Coinmarketcap
- coindfpull: pulls coin information from cmc including total coin supply
- tokendfpull: pulls token information from cmc including total token supply
- coincircdf: pulls coin circulating supply from cmc
- tokencircdf: pulls token circulating supply from cmc

In [2]:
def coindfpull(pagenum):
    site = 'https://coinmarketcap.com/coins/views/market-cap-by-total-supply/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 

    ranks = []
    currnames = []
    mcapsusd = []
    mcapsbtc = []
    pricesusd = []
    pricesbtc = []
    volumes24husd = []
    volumes24hbtc = []
    totalsupplies = []
    #changes24husd = []
    #changes24hbtc = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            ranks = ranks + [columns[2].replace(' ','')]
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            mcapsusd = mcapsusd + [columns[9].split('data-btc="')[1].split(' " data-usd="')[1].replace('">','')]
            mcapsbtc = mcapsbtc + [columns[9].split('data-btc="')[1].split(' " data-usd="')[0]]
            pricesusd = pricesusd + [columns[15].split('" data-btc="')[1].split('" data-usd="')[1].split('" ')[0]]
            pricesbtc = pricesbtc + [columns[15].split('" data-btc="')[1].split('" data-usd="')[0]]
            volumes24husd = volumes24husd + [columns[18].split('" data-btc="')[1].split('" data-usd="')[1].split('" ')[0]]
            volumes24hbtc = volumes24hbtc + [columns[18].split('" data-btc="')[1].split('" data-usd="')[0]]
            totalsupplies = totalsupplies + [columns[21].split('data-supply="')[1].split('" ')[0].replace('">','')]
            #changes24husd = changes24husd + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[1].split('">')[0]]
            #changes24hbtc = changes24hbtc + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[0]]
    
    crypto_dict = {"Rank": ranks,
                   "Names": currnames,
                   "MktCapUSD": mcapsusd,
                   "MktCapBTC": mcapsbtc,
                   "PricesUSD": pricesusd,
                   "PricesBTC": pricesbtc,
                   "Vol24USD": volumes24husd,
                   "Vol24BTC": volumes24hbtc,
                   "TotalSupply": totalsupplies}
                   #"Change24USD": changes24husd,
                   #"Change24BTC": changes24hbtc
                  #}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    
    return outdf

def tokendfpull(pagenum):
    site = 'https://coinmarketcap.com/tokens/views/market-cap-by-total-supply/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 

    ranks = []
    currnames = []
    platforms = []
    mcapsusd = []
    mcapsbtc = []
    pricesusd = []
    pricesbtc = []
    volumes24husd = []
    volumes24hbtc = []
    totalsupplies = []
    #changes24husd = []
    #changes24hbtc = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            ranks = ranks + [columns[2].replace(' ','')]
            platforms = platforms + [columns[0].split('data-platformsymbol="')[1].split('" ')[0]]
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            mcapsusd = mcapsusd + [columns[10].split('data-btc="')[1].split(' " ')[1].split('data-usd="')[1].replace('">','')]
            mcapsbtc = mcapsbtc + [columns[10].split('data-btc="')[1].split(' " ')[0]]
            pricesusd = pricesusd + [columns[16].split('data-btc="')[1].split('" ')[2].replace('data-usd="','')]
            pricesbtc = pricesbtc + [columns[16].split('data-btc="')[1].split('" ')[0]]
            volumes24husd = volumes24husd + [columns[19].split('data-btc="')[1].split('" ')[2].replace('data-usd="','')]
            volumes24hbtc = volumes24hbtc + [columns[19].split('data-btc="')[1].split('" ')[0]]
            totalsupplies = totalsupplies + [columns[22].split('data-supply="')[1].split('" ')[0].replace('">','')]
            #changes24husd = changes24husd + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[1].split('">')[0]]
            #changes24hbtc = changes24hbtc + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[0].split('" ')[0]]
    
    crypto_dict = {"Rank": ranks,
                   "Names": currnames,
                   "Platform": platforms,
                   "MktCapUSD": mcapsusd,
                   "MktCapBTC": mcapsbtc,
                   "PricesUSD": pricesusd,
                   "PricesBTC": pricesbtc,
                   "Vol24USD": volumes24husd,
                   "Vol24BTC": volumes24hbtc,
                   "TotalSupply": totalsupplies}
                   #"Change24USD": changes24husd,
                   #"Change24BTC": changes24hbtc
                  #}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    
    return outdf

def coincircdf(pagenum):
    site = 'https://coinmarketcap.com/coins/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 
    
    currnames = []
    circulatingsupplies = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            circulatingsupplies = circulatingsupplies + [columns[21].split('data-supply="')[1].split('" ')[0].replace('">','')]
    
    crypto_dict = {"Names": currnames,
                   "CirculatingSupply": circulatingsupplies}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    del outdf['Names']
    
    return outdf

def tokencirdf(pagenum):
    site = 'https://coinmarketcap.com/tokens/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 
    
    currnames = []
    circulatingsupplies = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            circulatingsupplies = circulatingsupplies + [columns[22].split('data-supply="')[1].split('" ')[0].replace('">','')]
    
    crypto_dict = {"Names": currnames,
                   "CirculatingSupply": circulatingsupplies}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    del outdf['Names']
    
    return outdf

# Alternative Function for Pulling all Coin/Token Caps

In [17]:
def pullfulldf():
    site = 'https://coinmarketcap.com/all/views/all/'
    res = requests.get(site)
    soup = bs(requests.get(site).text, "lxml")

    table_start = '<table class="table js-summary-table dataTable no-footer" id="currencies-all" style="font-size: 14px; width: 100%;" role="grid">'
    table_end = '</table>'

    table = soup.find_all('table')[0]

    symbol = []
    cname = []
    mcapsusd = []
    pricesusd = []
    pricesbtc = []
    volumes24husd = []
    totalsupplies = []
    availablesupplies = []
    maxsupplies = []
    change1h = []
    change24h = []
    change7d = []
    i = 1
    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if not('class' in columns[0]):
            continue

        cmcname = columns[0].split('"id-')[1].replace('">','')

        apisite = 'https://api.coinmarketcap.com/v1/ticker/' + cmcname + '/'
        response = requests.get(apisite)
        json_data = json.loads(response.text)

        cname = cname + [cmcname]
        symbol = symbol + [json_data[0]['symbol']]
        mcapsusd = mcapsusd + [json_data[0]['market_cap_usd']]
        pricesusd = pricesusd + [json_data[0]['price_usd']]
        pricesbtc = pricesbtc + [json_data[0]['price_btc']]
        volumes24husd = volumes24husd + [json_data[0]['24h_volume_usd']]
        totalsupplies = totalsupplies + [json_data[0]['total_supply']]
        availablesupplies = availablesupplies + [json_data[0]['available_supply']]
        maxsupplies = maxsupplies + [json_data[0]['max_supply']]
        change1h = change1h + [json_data[0]['percent_change_1h']]
        change24h = change24h + [json_data[0]['percent_change_24h']]
        change7d = change7d + [json_data[0]['percent_change_7d']]
        
        print('Number: ' + str(i) + ' Coin: ' + json_data[0]['symbol'])
        sleep(1)
        i += 1

    fulldict = {'CMCname': cname,
                'Symbol': symbol,
                'Mcap': mcapsusd,
                'USDPrice': pricesusd,
                'BTCPrice': pricesbtc,           
                'Vol24h': volumes24husd,           
                'TotalSupply': totalsupplies,           
                'AvailableSupply': availablesupplies,    
                'MaxSupply': maxsupplies,
                'Change1h': change1h,
                'Change24h': change24h,
                'Change7d': change7d}
    fulldf = pd.DataFrame(fulldict,index = cname)
    
    return fulldf


# Creating dataframe for initial filtering
- Keep only marketcaps with less than 100M
- Remove tokens with 24 hour volumes less than 2% of marketcaps
- Remove tokens with circulating supply/total supply with less than 20%

In [10]:
# Creating DF for Coins

def microfilter1(mcapthreshold, volthreshold, supplythreshold):
    pagenmbr = 1
    coinlist = []
    while type(coindfpull(pagenmbr)) != str:
        coinlist = coinlist + [coindfpull(pagenmbr)]
        pagenmbr += 1

    pagenmbr = 1
    coincirclist = []
    while type(coincircdf(pagenmbr)) != str:
        coincirclist = coincirclist + [coincircdf(pagenmbr)]
        pagenmbr += 1

    coinscirc = pd.concat(coincirclist, axis = 0)
    coins = pd.concat(coinlist, axis = 0)
    coins = coins[coins['MktCapUSD'] != '?']
    coins = pd.concat([coins.drop_duplicates(),coinscirc.drop_duplicates()], axis = 1, join = 'inner')

    # Creating DF for Tokens

    pagenmbr = 1
    tokenlist = []
    while type(tokendfpull(pagenmbr)) != str:
        tokenlist = tokenlist + [tokendfpull(pagenmbr)]
        pagenmbr += 1

    pagenmbr = 1
    tokencirclist = []
    while type(tokencirdf(pagenmbr)) != str:
        tokencirclist = tokencirclist + [tokencirdf(pagenmbr)]
        pagenmbr += 1

    tokenscirc = pd.concat(tokencirclist, axis = 0)
    tokens = pd.concat(tokenlist, axis = 0)
    tokens = tokens[tokens['MktCapUSD'] != '?']
    tokens = pd.concat([tokens.drop_duplicates(),tokenscirc.drop_duplicates()], axis = 1, join = 'inner')

    totaldf = pd.concat([coins,tokens], axis = 0)
    totaldf = totaldf[totaldf['CirculatingSupply'] != 'None']
    totaldf = totaldf[totaldf['Vol24USD'] != 'None']

    totaldf['MktCapUSD'] = pd.to_numeric(totaldf['MktCapUSD'])
    totaldf['CirculatingSupply'] = pd.to_numeric(totaldf['CirculatingSupply'])
    totaldf['TotalSupply'] = pd.to_numeric(totaldf['TotalSupply'])
    totaldf['Vol24USD'] = pd.to_numeric(totaldf['Vol24USD'])
    totaldf['VolRatio'] = totaldf['Vol24USD']/totaldf['MktCapUSD']
    totaldf['SupplyRatio'] = totaldf['CirculatingSupply']/totaldf['TotalSupply']
    
    totaldf = totaldf[totaldf['MktCapUSD'] <= mcapthreshold]
    totaldf = totaldf[totaldf['VolRatio'] >= volthreshold]
    totaldf = totaldf[totaldf['SupplyRatio'] >= supplythreshold]
    
    
    return totaldf

In [11]:
def checkannouncement(cmcname):

    site = 'https://coinmarketcap.com/currencies/' + cmcname + '/'

    res = requests.get(site)
    soup = str(bs(res.text, 'lxml'))

    if "Announcement" in soup:
        announcementsite = soup.split('<li><span class="glyphicon glyphicon-bullhorn text-gray" title="Announcement">')[1].split('<a href="')[1].split('" target')[0]
    else:
        announcementsite = "NA"

    if 'bitcointalk.org' in announcementsite:
        return 1, announcementsite
    else:
        return 0, 'NA'
    
def bctcheck(cmcname):

    annsite = checkannouncement(cmcname)[1]
    parsesoup = annsite.split('=')[0] + '=' + annsite.split('=')[1].split('.')[0]
    res = requests.get(parsesoup)
    soup = str(bs(res.text, 'lxml'))
    discussion = soup.split('<td class="middletext"')[1].split('<span class="prevnext">')[0]
    discussion = discussion.split('<div class="nav"')[0]
    if 'All' in discussion.split(parsesoup)[-1]:
        lastdiscuss = discussion.split(parsesoup)[-2]
    else:
        lastdiscuss = discussion.split(parsesoup)[-1]
        
    if '<td align="right" style="padding-right:' in discussion:
        lastpgIndex = lastdiscuss.split('">')[0]
    else:
        lastpgIndex = lastdiscuss.split('">')[0]

    lastpg = parsesoup + lastpgIndex
    res = requests.get(lastpg)
    soup = str(bs(res.text, 'lxml'))
    totpages = soup.split('<td class="middletext"')[1].split('[<b>')[1].split('</b>]')[0]

    forumparse = lastpg.split('=')[1]
    res = requests.get(lastpg)
    soup = str(bs(res.text, 'lxml'))
    lastmsg = soup.split(forumparse)[1].split('<tbody>')[-1].split('<div class="smalltext">')[-1].split('</div>')[0]
    
    return totpages, lastmsg

In [ ]:
'''
# Using old method to build dataframe, no change fields available

cmcdf = microfilter1(100000000, 0.02, 0.1)

names = list(cmcdf.index)
announcements = []
bctpages = []
lastentries = []

for cmcname in names:
    bctentry, annsite = checkannouncement(cmcname)
    announcements = announcements + [bctentry]
    if bctentry == 1:
        totpgs, last = bctcheck(cmcname)
    else:
        totpgs, last = (0,0)
        
    bctpages = bctpages + [totpgs]
    lastentries = lastentries + [last]
    print('Name: ' + str(len(bctpages)) + ' Alt: ' + cmcname)
    sleep(1)
    
bctdf = pd.DataFrame({'bctpages': bctpages,
                     'lastentry': lastentries},
                    index = names)
filterdf = pd.concat([cmcdf,bctdf],axis = 1)
filterdf.to_csv('cmc2.csv')

'''

In [45]:
cmcfull = pullfulldf()

names = list(cmcfull.index)
announcements = []
bctpages = []
lastentries = []

for cmcname in names:
    bctentry, annsite = checkannouncement(cmcname)
    announcements = announcements + [bctentry]
    if bctentry == 1:
        try:
            totpgs, last = bctcheck(cmcname)
        except:
            totpgs, last = (0,0)
        
    else:
        totpgs, last = (0,0)
        
    bctpages = bctpages + [totpgs]
    lastentries = lastentries + [last]
    print('Name: ' + str(len(bctpages)) + ' Alt: ' + cmcname)
    sleep(1)
    
bctdf = pd.DataFrame({'bctpages': bctpages,
                     'lastentry': lastentries},
                    index = names)
filterdf = pd.concat([cmcfull,bctdf],axis = 1)

binance = pd.read_csv('binancelist.csv')
binance.index = binance['Name']
del binance['Ticker'], binance['Name']

bittrex = pd.read_excel('bittrexlist.xlsx')
bittrex.index = bittrex['Name']
del bittrex['Ticker'], bittrex['Name']

kucoin = pd.read_csv('kucoinlist.csv')
kucoin.index = kucoin['Name']
del kucoin['Ticker'], kucoin['Name']

cryptopia = pd.read_csv('cryptopialist.csv')
cryptopia.index = cryptopia['Name']
del cryptopia['Ticker'], cryptopia['Name']

fulldf = pd.concat([filterdf,cryptopia,kucoin,binance,bittrex],axis = 1)
fulldf.head()

fulldf.to_csv('cmc.csv')

Number: 1 Coin: BTC
Number: 2 Coin: ETH
Number: 3 Coin: XRP
Number: 4 Coin: BCH
Number: 5 Coin: ADA
Number: 6 Coin: XEM
Number: 7 Coin: LTC
Number: 8 Coin: XLM
Number: 9 Coin: MIOTA
Number: 10 Coin: TRX
Number: 11 Coin: DASH
Number: 12 Coin: NEO
Number: 13 Coin: XMR
Number: 14 Coin: EOS
Number: 15 Coin: ICX
Number: 16 Coin: QTUM
Number: 17 Coin: BTG
Number: 18 Coin: XRB
Number: 19 Coin: ETC
Number: 20 Coin: LSK
Number: 21 Coin: XVG
Number: 22 Coin: OMG
Number: 23 Coin: SC
Number: 24 Coin: BCN
Number: 25 Coin: BCC
Number: 26 Coin: ZEC
Number: 27 Coin: PPT
Number: 28 Coin: STRAT
Number: 29 Coin: DCN
Number: 30 Coin: BTS
Number: 31 Coin: BNB
Number: 32 Coin: ARDR
Number: 33 Coin: KCS
Number: 34 Coin: USDT
Number: 35 Coin: DOGE
Number: 36 Coin: SNT
Number: 37 Coin: STEEM
Number: 38 Coin: VEN
Number: 39 Coin: WAVES
Number: 40 Coin: WAX
Number: 41 Coin: DRGN
Number: 42 Coin: DGB
Number: 43 Coin: ZRX
Number: 44 Coin: ARK
Number: 45 Coin: REP
Number: 46 Coin: HSR
Number: 47 Coin: DENT
Number: 

Number: 372 Coin: CAT
Number: 373 Coin: ITNS
Number: 374 Coin: EBST
Number: 375 Coin: OPT
Number: 376 Coin: XVC
Number: 377 Coin: MYB
Number: 378 Coin: CURE
Number: 379 Coin: BITUSD
Number: 380 Coin: LUX
Number: 381 Coin: BSD
Number: 382 Coin: RMC
Number: 383 Coin: CVCOIN
Number: 384 Coin: BCY
Number: 385 Coin: SPR
Number: 386 Coin: BWK
Number: 387 Coin: WILD
Number: 388 Coin: OBITS
Number: 389 Coin: DOT
Number: 390 Coin: PIX
Number: 391 Coin: BBR
Number: 392 Coin: BTM
Number: 393 Coin: ZNY
Number: 394 Coin: ABY
Number: 395 Coin: FRST
Number: 396 Coin: CSNO
Number: 397 Coin: ATMS
Number: 398 Coin: NOBL
Number: 399 Coin: 1337
Number: 400 Coin: XPM
Number: 401 Coin: QWARK
Number: 402 Coin: NTRN
Number: 403 Coin: CHC
Number: 404 Coin: UNO
Number: 405 Coin: SPHR
Number: 406 Coin: SSS
Number: 407 Coin: PLU
Number: 408 Coin: NVST
Number: 409 Coin: CANN
Number: 410 Coin: PND
Number: 411 Coin: DRP
Number: 412 Coin: DOPE
Number: 413 Coin: CARBON
Number: 414 Coin: PRIX
Number: 415 Coin: TX
Numbe

Number: 738 Coin: ETG
Number: 739 Coin: BUCKS
Number: 740 Coin: LBTC
Number: 741 Coin: SDRN
Number: 742 Coin: UNI
Number: 743 Coin: BTCRED
Number: 744 Coin: 888
Number: 745 Coin: HXX
Number: 746 Coin: BXT
Number: 747 Coin: VLT
Number: 748 Coin: XRE
Number: 749 Coin: LCP
Number: 750 Coin: DSH
Number: 751 Coin: UNIC
Number: 752 Coin: ARCO
Number: 753 Coin: NTWK
Number: 754 Coin: TSE
Number: 755 Coin: FNC
Number: 756 Coin: ITI
Number: 757 Coin: EUC
Number: 758 Coin: EVIL
Number: 759 Coin: XGR
Number: 760 Coin: MNM
Number: 761 Coin: GRIM
Number: 762 Coin: BLC
Number: 763 Coin: TAG
Number: 764 Coin: JET
Number: 765 Coin: BCF
Number: 766 Coin: EMD
Number: 767 Coin: SPACE
Number: 768 Coin: MOJO
Number: 769 Coin: VOT
Number: 770 Coin: BSTY
Number: 771 Coin: EGAS
Number: 772 Coin: MARS
Number: 773 Coin: ABJ
Number: 774 Coin: CUBE
Number: 775 Coin: TGC
Number: 776 Coin: PR
Number: 777 Coin: UNY
Number: 778 Coin: GAP
Number: 779 Coin: POST
Number: 780 Coin: KED
Number: 781 Coin: DUO
Number: 782 C

Number: 1096 Coin: TSL
Number: 1097 Coin: SMT
Number: 1098 Coin: CAN
Number: 1099 Coin: QLC
Number: 1100 Coin: BCD
Number: 1101 Coin: QBT
Number: 1102 Coin: CAT
Number: 1103 Coin: SPHTX
Number: 1104 Coin: HTML
Number: 1105 Coin: HPY
Number: 1106 Coin: MDT
Number: 1107 Coin: ENT
Number: 1108 Coin: CL
Number: 1109 Coin: WIC
Number: 1110 Coin: SBTC
Number: 1111 Coin: TAU
Number: 1112 Coin: PRO
Number: 1113 Coin: SHND
Number: 1114 Coin: DEW
Number: 1115 Coin: FRGC
Number: 1116 Coin: IGNIS
Number: 1117 Coin: BCX
Number: 1118 Coin: BIG
Number: 1119 Coin: CAPP
Number: 1120 Coin: NGC
Number: 1121 Coin: COUPE
Number: 1122 Coin: MKR
Number: 1123 Coin: XTZ
Number: 1124 Coin: AI
Number: 1125 Coin: ACE
Number: 1126 Coin: WC
Number: 1127 Coin: DIM
Number: 1128 Coin: B2X
Number: 1129 Coin: UGT
Number: 1130 Coin: CMS
Number: 1131 Coin: VIU
Number: 1132 Coin: CYDER
Number: 1133 Coin: INF
Number: 1134 Coin: LLT
Number: 1135 Coin: XIN
Number: 1136 Coin: GBG
Number: 1137 Coin: HBC
Number: 1138 Coin: CLUB


Name: 50 Alt: golem-network-tokens
Name: 51 Alt: basic-attention-token
Name: 52 Alt: salt
Name: 53 Alt: electroneum
Name: 54 Alt: decred
Name: 55 Alt: funfair
Name: 56 Alt: pivx
Name: 57 Alt: kyber-network
Name: 58 Alt: medibloc
Name: 59 Alt: experience-points
Name: 60 Alt: ethos
Name: 61 Alt: qash
Name: 62 Alt: substratum
Name: 63 Alt: aion
Name: 64 Alt: reddcoin
Name: 65 Alt: kin
Name: 66 Alt: nexus
Name: 67 Alt: aelf
Name: 68 Alt: factom
Name: 69 Alt: power-ledger
Name: 70 Alt: request-network
Name: 71 Alt: aeternity
Name: 72 Alt: gas
Name: 73 Alt: neblio
Name: 74 Alt: bytom
Name: 75 Alt: iconomi
Name: 76 Alt: enigma-project
Name: 77 Alt: deepbrain-chain
Name: 78 Alt: monacoin
Name: 79 Alt: maidsafecoin
Name: 80 Alt: nxt
Name: 81 Alt: bitcoindark
Name: 82 Alt: rchain
Name: 83 Alt: byteball
Name: 84 Alt: storm
Name: 85 Alt: cobinhood
Name: 86 Alt: syscoin
Name: 87 Alt: digixdao
Name: 88 Alt: digitalnote
Name: 89 Alt: gnosis-gno
Name: 90 Alt: chainlink
Name: 91 Alt: santiment
Name: 92

Name: 391 Alt: boolberry
Name: 392 Alt: bitmark
Name: 393 Alt: bitzeny
Name: 394 Alt: artbyte
Name: 395 Alt: firstcoin
Name: 396 Alt: bitdice
Name: 397 Alt: atmos
Name: 398 Alt: noblecoin
Name: 399 Alt: 1337coin
Name: 400 Alt: primecoin
Name: 401 Alt: qwark
Name: 402 Alt: neutron
Name: 403 Alt: chaincoin
Name: 404 Alt: unobtanium
Name: 405 Alt: sphere
Name: 406 Alt: sharechain
Name: 407 Alt: pluton
Name: 408 Alt: nvo
Name: 409 Alt: cannabiscoin
Name: 410 Alt: pandacoin-pnd
Name: 411 Alt: dcorp
Name: 412 Alt: dopecoin
Name: 413 Alt: carboncoin
Name: 414 Alt: privatix
Name: 415 Alt: transfercoin
Name: 416 Alt: altcoin-alt
Name: 417 Alt: crowdcoin
Name: 418 Alt: b2bx
Name: 419 Alt: leocoin
Name: 420 Alt: oxycoin
Name: 421 Alt: monacocoin
Name: 422 Alt: exchange-union
Name: 423 Alt: breakout
Name: 424 Alt: buzzcoin
Name: 425 Alt: etheroll
Name: 426 Alt: zeitcoin
Name: 427 Alt: bitcoinz
Name: 428 Alt: procurrency
Name: 429 Alt: hyperstake
Name: 430 Alt: royal-kingdom-coin
Name: 431 Alt: pes

Name: 723 Alt: trackr
Name: 724 Alt: hicoin
Name: 725 Alt: coin2-1
Name: 726 Alt: berncash
Name: 727 Alt: trident
Name: 728 Alt: x-coin
Name: 729 Alt: paycoin2
Name: 730 Alt: digital-developers-fund
Name: 731 Alt: chancoin
Name: 732 Alt: btsr
Name: 733 Alt: sterlingcoin
Name: 734 Alt: litebar
Name: 735 Alt: pioneer-coin
Name: 736 Alt: coin
Name: 737 Alt: sooncoin
Name: 738 Alt: ethereum-gold
Name: 739 Alt: swagbucks
Name: 740 Alt: litebitcoin
Name: 741 Alt: senderon
Name: 742 Alt: universe
Name: 743 Alt: bitcoin-red
Name: 744 Alt: octocoin
Name: 745 Alt: hexx
Name: 746 Alt: bittokens
Name: 747 Alt: veltor
Name: 748 Alt: revolvercoin
Name: 749 Alt: litecoin-plus
Name: 750 Alt: dashcoin
Name: 751 Alt: unicoin
Name: 752 Alt: aquariuscoin
Name: 753 Alt: network-token
Name: 754 Alt: tattoocoin
Name: 755 Alt: fincoin
Name: 756 Alt: iticoin
Name: 757 Alt: eurocoin
Name: 758 Alt: evil-coin
Name: 759 Alt: goldreserve
Name: 760 Alt: mineum
Name: 761 Alt: grimcoin
Name: 762 Alt: blakecoin
Name: 7

Name: 1046 Alt: gamebet-coin
Name: 1047 Alt: zayedcoin
Name: 1048 Alt: flavorcoin
Name: 1049 Alt: rsgpcoin
Name: 1050 Alt: destiny
Name: 1051 Alt: litecred
Name: 1052 Alt: arbit
Name: 1053 Alt: morningstar-payments
Name: 1054 Alt: mtmgaming
Name: 1055 Alt: bowscoin
Name: 1056 Alt: genstake
Name: 1057 Alt: impulsecoin
Name: 1058 Alt: orlycoin
Name: 1059 Alt: posex
Name: 1060 Alt: tagrcoin
Name: 1061 Alt: crtcoin
Name: 1062 Alt: high-voltage
Name: 1063 Alt: speedcash
Name: 1064 Alt: xonecoin
Name: 1065 Alt: kingn-coin
Name: 1066 Alt: letitride
Name: 1067 Alt: cabbage
Name: 1068 Alt: osmiumcoin
Name: 1069 Alt: cryptoescudo
Name: 1070 Alt: antilitecoin
Name: 1071 Alt: biobar
Name: 1072 Alt: qibuck-asset
Name: 1073 Alt: frazcoin
Name: 1074 Alt: gbcgoldcoin
Name: 1075 Alt: nodecoin
Name: 1076 Alt: ccminer
Name: 1077 Alt: agrolifecoin
Name: 1078 Alt: sydpak
Name: 1079 Alt: save-and-gain
Name: 1080 Alt: burstocean
Name: 1081 Alt: lex4all
Name: 1082 Alt: sojourn
Name: 1083 Alt: pizzacoin
Name: 

Name: 1362 Alt: richcoin
Name: 1363 Alt: snakeeyes
Name: 1364 Alt: mmxvi
Name: 1365 Alt: deltacredits
Name: 1366 Alt: xaucoin
Name: 1367 Alt: shellcoin
Name: 1368 Alt: xde-ii
Name: 1369 Alt: tattoocoin-limited
Name: 1370 Alt: trickycoin
Name: 1371 Alt: kashhcoin
Name: 1372 Alt: gameleaguecoin
Name: 1373 Alt: mobilecash
Name: 1374 Alt: qora
Name: 1375 Alt: psilocybin
Name: 1376 Alt: rhfcoin
Name: 1377 Alt: international-diamond
Name: 1378 Alt: cbd-crystals
Name: 1379 Alt: dubstep
Name: 1380 Alt: goldunioncoin
Name: 1381 Alt: teracoin
Name: 1382 Alt: batcoin
Name: 1383 Alt: rabbitcoin
Name: 1384 Alt: futcoin
Name: 1385 Alt: ocow
Name: 1386 Alt: 9coin
Name: 1387 Alt: aidos-kuneen
Name: 1388 Alt: royalties
Name: 1389 Alt: embercoin
Name: 1390 Alt: xylo
Name: 1391 Alt: santa-coin
Name: 1392 Alt: madcoin
Name: 1393 Alt: casinocoin
Name: 1394 Alt: bitcoin-unlimited
Name: 1395 Alt: faceblock
Name: 1396 Alt: hyper-tv
Name: 1397 Alt: qcash
Name: 1398 Alt: farstcoin
